<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# pandas Special

Dr. Yves J. Hilpisch

The Python Quants GmbH

<a href='http://fpq.io'>http://fpq.io</a> | <a href='mailto:team@tpq.io'>team@tpq.io</a>

## Some Special Analytics Topics

### Performance Revisited

A **sample data** set to work with.

In [ ]:
from sys import version_info
version_info

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import warnings; warnings.simplefilter('ignore')

In [ ]:
data = np.random.standard_normal((10000000, 2))
data.nbytes

In [ ]:
df = pd.DataFrame(data, columns=['x', 'y'])

**First** data rows.

In [ ]:
df.head()

Performance of simple **arithmetic operations** (I).

In [ ]:
%time res = df['x'] + df['y']
res[:5]

Performance of simple **arithmetic operations** (II).

In [ ]:
%time res = df.sum(axis=1)
res[:5]

In [ ]:
%time res = df.values.sum(axis=1)
res[:5]

Performance of simple **arithmetic operations** (III).

In [ ]:
%time res = np.sum(df, axis=1)
res[:5]

In [ ]:
%time res = np.sum(df.values, axis=1)
res[:5]

Performance of simple **arithmetic operations** (IV).

In [ ]:
%time res = df.eval('x + y')
res[:5]

### Complex Data Selection

The data **visualized**.

In [ ]:
%matplotlib inline
df[:1000].plot(x='x', y='y', kind='scatter')

Selecting via **indexing** (I).

In [ ]:
%time res = df[df['x'] > 4.5]
res[:5]

Selecting via **indexing** (II).

In [ ]:
%time res = df[(df['x'] > 4.5) | (df['y'] < -4.5)]
res[:5]

Selecting via **indexing** (III).

In [ ]:
%matplotlib inline
df[(df['x'] > 2.5) | (df['y'] < -2.5)][:1000].plot(
                        x='x', y='y', kind='scatter')

Selecting via **indexing** (IV).

In [ ]:
df[(df.x > 2.5) & (df.y < -2.5)][:1000].plot(x='x', y='y', kind='scatter')

## More on Visualization

The **sample data** to plot.

In [ ]:
np.random.seed(1000)
data = np.random.standard_normal((500, 2)).cumsum(axis=0)
index = pd.date_range(start='2015-1-1', periods=len(data), freq='B')

In [ ]:
df = pd.DataFrame(data, index=index, columns=['A', 'B'])

The `plot` method(s) of `pandas` allow for rather **customized plots**. First, the default.

In [ ]:
df.plot()

Some **customizations**.

In [ ]:
df.plot(subplots=True, color='r');

Some **customizations**.

In [ ]:
df.plot(legend=False, title='Custom Plot')

Some **customizations**.

In [ ]:
df.plot(style=['r.', 'm-.'])

Some **customizations**.

In [ ]:
(df ** 4).plot(logy=True)

Some **customizations**.

In [ ]:
df['B'] = df['B'] * 100

In [ ]:
df.plot(secondary_y='B', grid=True, figsize=(10, 5))

## I/O with pandas

One of the major strengths of the `pandas` library is that it can **read and write different data formats natively**, among others:

* `CSV` (comma separated value)
* `SQL` (structured query language)
* `XLS/XSLX` (Microsoft Excel files)
* `JSON` (JavaScript object notation)
* `HTML` (hypertext markup language)

Our test case is again a **large set of floating point numbers** (1mn rows).

In [ ]:
import numpy as np
import pandas as pd
data = np.random.standard_normal((1000000, 5)).round(5)
        # sample data set

In [ ]:
try:
    !mkdir data
except:
    pass

In [ ]:
# replace "yves" by your unique user name
# AND create a folder "data" in your home directory
path = 'data/'
filename = path + 'numbs'

In [ ]:
import os
os.environ['path'] = path

### SQL Database

The **benchmark case** is based on `SQLite3`.

In [ ]:
import sqlite3 as sq3

In [ ]:
query = 'CREATE TABLE numbers (No1 real, No2 real,\
        No3 real, No4 real, No5 real)'

In [ ]:
con = sq3.Connection(filename + '.db')

In [ ]:
con.execute(query)

**Writing the data** in bulk.

In [ ]:
%%time
con.executemany('INSERT INTO numbers VALUES (?, ?, ?, ?, ?)', data)
con.commit()

In [ ]:
ll $path

**Reading is faster** than writing.

In [ ]:
%%time
temp = con.execute('SELECT * FROM numbers').fetchall()
print (temp[:2])
temp = 0.0

Reading a `SQL` query result into a **`ndarray` object** ...

In [ ]:
%%time
query = 'SELECT * FROM numbers WHERE No1 > 0 AND No2 < 0'
res = np.array(con.execute(query).fetchall()).round(3)

... and **plotting** it.

In [ ]:
res = res[::100]  # every 100th result
import matplotlib.pyplot as plt
plt.plot(res[:, 0], res[:, 1], 'ro')
plt.grid(True); plt.xlim(-0.5, 4.5); plt.ylim(-4.5, 0.5)

### From SQL to pandas

`pandas` can be used to make such an operation more **convenient and efficient**.

In [ ]:
# import pandas.io.sql as pds

The code for reading the data becomes a bit **more compact**.

In [ ]:
%time data = pd.read_sql('SELECT * FROM numbers', con)
con.close()

In [ ]:
data.head()

The data is now **in-memory**. This allows for much **faster analytics**.

In [ ]:
%time data[(data['No1'] > 0) & (data['No2'] < 0)].head()

A more **complex query**.

In [ ]:
%%time
res = data[['No1', 'No2']][((data['No1'] > 0.5) | (data['No1'] < -0.5))
                     & ((data['No2'] < -1) | (data['No2'] > 1))]

In [ ]:
plt.plot(res.No1, res.No2, 'ro')
plt.grid(True); plt.axis('tight')

**EXERCISE**: Implement a similar query and plot in 3-d.

**Writing the data** to disk with `pandas`.

In [ ]:
h5s = pd.HDFStore(filename + '.h5s', 'w')

In [ ]:
%time h5s['data'] = data

In [ ]:
h5s

In [ ]:
h5s.close()

Again, **reading is even faster**.

In [ ]:
%%time
h5s = pd.HDFStore(filename + '.h5s', 'r')
temp = h5s['data']
h5s.close()

A brief check whether the data sets are indeed the same.

In [ ]:
np.allclose(np.array(temp), np.array(data))

In [ ]:
temp = 0.0

Also a look at the two files now on disk, showing that the +HDF5+ format consumes somewhat less disk space.

In [ ]:
ll $path

### Data as CSV File

`pandas` is pretty good at **processing `CSV` files**.

In [ ]:
%time data.to_csv(filename + '.csv')

In [ ]:
%%time
pd.read_csv(filename + '.csv')[['No1', 'No2',
                                'No3', 'No4']].hist(bins=20);

### Data as Excel File

The same holds true for **Excel spreadsheet files** &ndash; however, performance is not too good with this format.

In [ ]:
%time data[:10000].to_excel(filename + '.xlsx')

In [ ]:
%time pd.read_excel(filename + '.xlsx', 'Sheet1').cumsum().plot()

Comparing **file sizes**.

In [ ]:
ll $path*

In [ ]:
!rm -f $path/*

### Using Compression

As usual, let us generate a **sample data set** to work with.

In [ ]:
1e6

In [ ]:
%%time
data = pd.DataFrame(np.random.randint(0, 100, (1e6, 5)))
data = pd.merge(data, pd.DataFrame(np.random.standard_normal((1e6, 5))),
                left_index=True, right_index=True)
  # integers and floats --> medium benefits from compression expected

Calling `.info()`.

In [ ]:
data.info()

A function to measure the **performance of compression** (time, compression ratio).

In [ ]:
import os
from time import time
def benchmarking(lib):
    times = []
    sizes = []
    for c in range(10):
        t0 = time()
        name = path + 'data.h5c%s' % c
        h5 = pd.HDFStore(name, complevel=c, complib=lib)
        h5['data'] = data
        h5.close()
        times.append(time() - t0)
        sizes.append(os.path.getsize(name))
    return times, sizes

Function to **plot the results**.

In [ ]:
# function to plot the results
def plot_results(times, sizes):
    fig, ax1 = plt.subplots()
    plt.plot(range(10), times, 'r', lw=1.5, label='time')
    plt.xlabel('comp level')
    plt.ylabel('time [sec]')
    plt.legend(loc=0)
    ax2 = ax1.twinx()
    plt.plot(range(10), sizes, 'g', lw=1.5, label='size')
    plt.ylabel('file size [bytes]')
    plt.legend(loc=7)
    plt.grid(True)

A **benchmarking**.

In [ ]:
times, sizes = benchmarking('blosc')  # zlib

In [ ]:
plot_results(times, sizes)

In [ ]:
!ls -an $path

In [ ]:
!rm $path/*

**EXERCISE**: Compare the performance of all compression libs available in `PyTables`.

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:yves@tpq.io">yves@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="http://hilpisch.com" target="_blank">http://hilpisch.com</a> 

**Quant Platform** &mdash; <a href="http://quant-platform.com" target="_blank">http://quant-platform.com</a>

**Python for Finance** &mdash; <a href="http://python-for-finance.com" target="_blank">http://python-for-finance.com</a>

**Derivatives Analytics with Python** &mdash; <a href="http://derivatives-analytics-with-python.com" target="_blank">http://derivatives-analytics-with-python.com</a>

**Python Trainings** &mdash; <a href="http://training.tpq.io" target="_blank">http://training.tpq.io</a>